### Задача 48 "Ортогональные латинские квадраты"
Сформулируйте задачу построения ортогональных латинских квадратов как задачу целочисленного(булева) линейного программирования. Неизвестными задачи будут булевы переменные Xijkl, где индексы i, j, k, l меняются от 1 до n, причем Xijkl <=> в позиции (i,j) стоит пара (k,l).
Задача имеет следующие ограничения :
Запишите условия задачи, используя язык моделирования, например, из библиотеки pulp или другой подобной. Решите задачу средствами библиотеки для n = 2, 3, 4, 5, 6,10. Сколько времени тратит ваша программа?


In [ ]:
!pip install pulp
from datetime import datetime
from pulp import *

for n in range(2, 11):
    # Инициализируем модель
    model = pulp.LpProblem(name="orthogonal_latin_squares", sense=LpMinimize)

    # Создаем переменные
    X = pulp.LpVariable.dicts("X", [(i, j, k, l) for i in range(1, n + 1)
                                         for j in range(1, n + 1)
                                         for k in range(1, n + 1)
                                         for l in range(1, n + 1)], cat="Binary")


    # Целевая функция
    model += 0, "Dummy Objective"

    # Ограничения для каждой клетки
    for i in range(1, n+1):
        for j in range(1, n+1):
            model += pulp.lpSum(X[i,j,k,l] for k in range(1, n+1) for l in range(1, n+1)) == 1, f"Cell ({i}, {j})"

    # Ограничения на элементы в строках
    for i in range(1, n+1):
        for k in range(1, n+1):
            model += pulp.lpSum(X[i,j,k,l] for j in range(1, n+1) for l in range(1, n+1)) == 1, f"Row {i}, Symbol {k}"

    # Ограничения на элементы в столбцах
    for j in range(1, n+1):
        for l in range(1, n+1):
            model += pulp.lpSum(X[i,j,k,l] for i in range(1, n+1) for k in range(1, n+1)) == 1, f"Column {j}, Symbol {l}"        
            
    # Ограничения на ортогональность квадратов
    for k in range(1, n+1):
        for l in range(1, n+1):
            for i in range(1, n+1):
                model += pulp.lpSum(X[i,j,k,l] for j in range(1, n+1)) == pulp.lpSum(X[i,j,l,k] for j in range(1, n+1)), f"Orthogonality {k}, {l}, Row {i}"
            for j in range(1, n+1):
                model += pulp.lpSum(X[i,j,k,l] for i in range(1, n+1)) == pulp.lpSum(X[i,j,l,k] for i in range(1, n+1)), f"Orthogonality {k}, {l}, Column {j}"
  
    # Решаем задачу
    start_time = datetime.now()
    model.solve()
    end_time = datetime.now()
    
    # Выводим результаты
    print(f"Решение для n = {n}")
    if model.status == LpStatusOptimal and n != 2 and n != 6 :
        print("Оптимальное решение найдено")
        print(f"Время выполнения: {end_time - start_time} сек")
        print("Первый квадрат:")
        c = 0
        for i in range(1, n + 1):
            for j in range(1, n + 1):
                for k in range(1, n + 1):
                    for l in range(1, n + 1):
                        if X[(i, j, k, l)].value() == 1:
                            if c < n:
                                print(f"({k}, {l})", end=" ")
                                c += 1
                            else:
                                print("\n")
                                print(f"({k}, {l})", end=" ")
                                c = 1
                            
        print("\n")     
        print("Второй квадрат:")
        for i in range(1, n + 1):
            for j in range(1, n + 1):
                for k in range(1, n + 1):
                    for l in range(1, n + 1):
                        if X[(i, j, k, l)].value() == 1:
                            if c < n:
                                print(f"({l}, {k})", end=" ")
                                c += 1
                            else:
                                print("\n")
                                print(f"({l}, {k})", end=" ")
                                c = 1
        print("\n")
    else:
        print("Решение не найдено")
    print("------------------------------")

### Вывод: 
как мы видим, при увеличении n время выполнения программы возрастает в связи с увеличением как количества переменных, так и количества ограничений.
Например, для n = 10 время выполнения составляет 0:02:46.602860 сек,  количество переменных равно 10^4, а ограничений - 10^6.
Поэтому на выполнение программы для больших значений n требуется очень много времени.
